# Creating a POS Tagger

We can train a classifier to work out which suffixes are most informative for POS tagging. We can begin by finding out what the most common suffixes are

In [1]:
from nltk.corpus import brown
from nltk import FreqDist

suffix_f_dist = FreqDist()
for word in brown.words():
    word = word.lower()
    suffix_f_dist[word[-1:]] += 1
    suffix_f_dist[word[-2:]] += 1
    suffix_f_dist[word[-3:]] += 1

suffix_f_dist

FreqDist({'e': 202946, ',': 175002, '.': 152999, 's': 128722, 'd': 105687, 't': 94459, 'he': 92084, 'n': 87889, 'a': 74912, 'of': 72978, ...})

In [2]:
common_suffixes = [suffix for (suffix, _) in suffix_f_dist.most_common(100)]
common_suffixes[:10]

['e', ',', '.', 's', 'd', 't', 'he', 'n', 'a', 'of']

Next, we'll define a feature extractor function which checks a given word for these suffixes:

In [3]:
def pos_features(word):
    features = {}
    for suffix in common_suffixes:
        features['endswith({})'.format(suffix)] = word.lower().endswith(suffix)
    return features

pos_features('test')

{'endswith(e)': False,
 'endswith(,)': False,
 'endswith(.)': False,
 'endswith(s)': False,
 'endswith(d)': False,
 'endswith(t)': True,
 'endswith(he)': False,
 'endswith(n)': False,
 'endswith(a)': False,
 'endswith(of)': False,
 'endswith(the)': False,
 'endswith(y)': False,
 'endswith(r)': False,
 'endswith(to)': False,
 'endswith(in)': False,
 'endswith(f)': False,
 'endswith(o)': False,
 'endswith(ed)': False,
 'endswith(nd)': False,
 'endswith(is)': False,
 'endswith(on)': False,
 'endswith(l)': False,
 'endswith(g)': False,
 'endswith(and)': False,
 'endswith(ng)': False,
 'endswith(er)': False,
 'endswith(as)': False,
 'endswith(ing)': False,
 'endswith(h)': False,
 'endswith(at)': False,
 'endswith(es)': False,
 'endswith(or)': False,
 'endswith(re)': False,
 'endswith(it)': False,
 'endswith(``)': False,
 'endswith(an)': False,
 "endswith('')": False,
 'endswith(m)': False,
 'endswith(;)': False,
 'endswith(i)': False,
 'endswith(ly)': False,
 'endswith(ion)': False,
 'endsw

Now that we've defined our feature extractor, we can use it to train a new decision tree classifier:

In [4]:
tagged_words = brown.tagged_words(categories='news')
feature_sets = [(pos_features(n), g) for (n,g) in tagged_words]
feature_sets[0]

({'endswith(e)': True,
  'endswith(,)': False,
  'endswith(.)': False,
  'endswith(s)': False,
  'endswith(d)': False,
  'endswith(t)': False,
  'endswith(he)': True,
  'endswith(n)': False,
  'endswith(a)': False,
  'endswith(of)': False,
  'endswith(the)': True,
  'endswith(y)': False,
  'endswith(r)': False,
  'endswith(to)': False,
  'endswith(in)': False,
  'endswith(f)': False,
  'endswith(o)': False,
  'endswith(ed)': False,
  'endswith(nd)': False,
  'endswith(is)': False,
  'endswith(on)': False,
  'endswith(l)': False,
  'endswith(g)': False,
  'endswith(and)': False,
  'endswith(ng)': False,
  'endswith(er)': False,
  'endswith(as)': False,
  'endswith(ing)': False,
  'endswith(h)': False,
  'endswith(at)': False,
  'endswith(es)': False,
  'endswith(or)': False,
  'endswith(re)': False,
  'endswith(it)': False,
  'endswith(``)': False,
  'endswith(an)': False,
  "endswith('')": False,
  'endswith(m)': False,
  'endswith(;)': False,
  'endswith(i)': False,
  'endswith(ly)': 

In [5]:
from nltk import DecisionTreeClassifier
from nltk.classify import accuracy

cutoff = int(len(feature_sets) * 0.1)
train_set, test_set = feature_sets[cutoff:], feature_sets[:cutoff]

In [6]:
classifier = DecisionTreeClassifier.train(train_set) # NLTK is a teaching toolkit which is not really optimized for speed. Therefore, this may take forever. For speed, use scikit-learn for the classifiers.

In [7]:
accuracy(classifier, test_set)

0.6270512182993535

In [8]:
classifier.classify(pos_features('cats'))

'NNS'

In [9]:
classifier.pseudocode(depth=4)

"if endswith(the) == False: \n  if endswith(,) == False: \n    if endswith(s) == False: \n      if endswith(.) == False: return '.'\n      if endswith(.) == True: return '.'\n    if endswith(s) == True: \n      if endswith(is) == False: return 'PP$'\n      if endswith(is) == True: return 'BEZ'\n  if endswith(,) == True: return ','\nif endswith(the) == True: return 'AT'\n"

To improve the classifier, we can add contextual features:

```py
def pos_features(sentence, i): [1]
    features = {"suffix(1)": sentence[i][-1:],
                "suffix(2)": sentence[i][-2:],
                "suffix(3)": sentence[i][-3:]}
    if i == 0:
        features["prev-word"] = "<START>"
    else:
        features["prev-word"] = sentence[i-1]
    return features
```

Then, instead of working with tagged words, we work with tagged sentences:
```py
tagged_sents = brown.tagged_sents(categories='news')
```

We can then improve this further by adding more features such as `prev-tag` etc.